In [1]:
from cassie import CassieEnv
import numpy as np

import time

env = CassieEnv("walking")

qpos = np.copy(env.sim.qpos())

jpos = qpos[env.pos_idx]

fbpos = qpos[0:7]

# keep feet from intersecting ground
fbpos[2] += 0.2

env.render()

In [2]:
# Joint limits converted to radians from XML file

ROLL_LOW   = -0.26
ROLL_HIGH  =  0.39

YAW_LOW    = -0.39
YAW_HIGH   =  0.39

PITCH_LOW  = -0.87
PITCH_HIGH =  1.4

KNEE_LOW   = -2.86
KNEE_HIGH  = -0.64

FOOT_LOW   = -2.44
FOOT_HIGH  = -0.52

# achilles = .75 + pitch = 1
# not currently correct though
ACHILLES_LIMIT = 1.75

DELTA = 0.05#0.025 # resolution for IK data set
ITERS = 500

# ramp down joint angles
while jpos[2] > PITCH_LOW and jpos[3] > KNEE_LOW:
#     qpos = np.copy(env.sim.qpos())
#     if qpos[10] < ACHILLES_ROD_LOW:
#         print(qpos[2])
    
    jpos[2] -= DELTA
    jpos[7] -= DELTA
    
    jpos[3] -= DELTA
    jpos[8] -= DELTA
    
    env.set_joint_pos(jpos, fbpos, iters=ITERS)
    env.render()
    
    #time.sleep(1)
#input()
%matplotlib notebook

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

ax.axis("equal")

data = []
X = []
Y = []
Z = []

pitch = PITCH_LOW
while pitch < PITCH_HIGH:
    pitch += DELTA
    
    knee = KNEE_LOW
    while knee < KNEE_HIGH:
            knee += DELTA
            
            jpos[2] = pitch
            jpos[7] = pitch

            jpos[3] = knee
            jpos[8] = knee

            env.set_joint_pos(jpos, fbpos, iters=ITERS)
            
            footpos = np.zeros(6)
            env.sim.foot_pos(footpos)
            
            qpos = np.copy(env.sim.qpos())
            qpos = qpos[7:21]
            
            # get XZ foot position relative to center of mass
            footpos_relative = [footpos[0] - fbpos[0], footpos[2] - fbpos[2]]#,
                                #footpos[3] - fbpos[0], footpos[5] - fbpos[2]]
            
            data += [(footpos_relative, qpos)]
                        
            X += [footpos[0]]
            Y += [footpos[1]]
            Z += [footpos[2]]
            
            #if qpos[10] + qpos[2] < ACHILLES_LIMIT:
            #    continue 
                
            env.render()

    ax.plot(X, Y, Z)    
    X = []
    Y = []
    Z = []
    
    # ramp knee angle back down
    while jpos[3] > KNEE_LOW:
        jpos[3] -= DELTA
        jpos[8] -= DELTA
        
        env.set_joint_pos(jpos, fbpos, iters=ITERS)
        env.render()
        
plt.show()

<IPython.core.display.Javascript object>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler
import torch.functional as F

class MLP(nn.Module):
    def __init__(self,
                 num_inputs,
                 num_outputs,
                 nonlinearity="tanh"):

        super(MLP, self).__init__()

        hidden_dims = (16, 16)
        
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers += [nn.Linear(num_inputs, hidden_dims[0])]
        for l in range(len(hidden_dims) - 1):
            in_dim = hidden_dims[l]
            out_dim = hidden_dims[l + 1]
            self.hidden_layers += [nn.Linear(in_dim, out_dim)]
        
        self.out = nn.Linear(hidden_dims[-1], num_outputs)
        
        if nonlinearity == "relu":
            self.nonlinearity = torch.relu
        elif nonlinearity == "tanh":
            self.nonlinearity = torch.tanh
        else:
            raise NotImplementedError

    def forward(self, inputs):
        x = inputs
        for l in self.hidden_layers:
            x = self.nonlinearity(l(x))
        pred = self.out(x)
        
        return pred

x = torch.Tensor(np.array([e[0] for e in data]))
y = torch.Tensor(np.array([e[1] for e in data]))

model = MLP(2, 14, "tanh")
print("# of parameters: ", sum(p.numel() for p in model.parameters()))

print("# of data points: ", y.size(0))
    
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=1e-2)
for t in range(500):
    sampler = BatchSampler(
        SubsetRandomSampler(range(y.size(0))),
        batch_size=128,
        drop_last=True
    )

    for indices in sampler:
        indices = torch.LongTensor(indices)
        
        y_batch = y[indices]
        x_batch = x[indices]

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x_batch)

        # Compute and print loss
        loss = criterion(y_pred, y_batch)
        
        if t % 50 == 0:
            print(t, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


# of parameters:  558
# of data points:  2070
0 2289.076416015625
0 2088.5
0 2011.6400146484375
0 1843.992919921875
0 1613.654296875
0 1412.91259765625
0 1183.86279296875
0 1057.635986328125
0 850.84423828125
0 650.235595703125
0 483.61639404296875
0 382.5871887207031
0 362.42803955078125
0 300.2633056640625
0 336.79248046875
0 291.616943359375
50 1.9718674421310425
50 1.9400330781936646
50 2.017678737640381
50 2.0711660385131836
50 2.1381356716156006
50 2.5165863037109375
50 1.8323431015014648
50 2.6104555130004883
50 2.3183822631835938
50 2.2109487056732178
50 2.4582667350769043
50 2.218099594116211
50 1.7544485330581665
50 1.9406965970993042
50 2.3238093852996826
50 1.9024019241333008
100 1.162697196006775
100 1.2098395824432373
100 0.9305732846260071
100 1.0994576215744019
100 0.8935073018074036
100 1.024600863456726
100 0.8564743399620056
100 1.061978816986084
100 0.7888417840003967
100 1.030981421470642
100 0.9249280691146851
100 0.9189954400062561
100 0.7985913753509521
100 1.03

In [4]:
from scipy.interpolate import CubicSpline

def get_trajectory(peak, stride, phase, stance=0.15):
    """
    Foot and center of mass trajectory generator

    The center of mass trajectory is a line q(t) = (x(t), 0, z). 

    The foot trajectories are periodic 3 point cubic splines in z(t),
    lines in x(t), and constant in y => q(t) = (x(t), y, z(t))

    Left and right foot trajectories are offset in time by "phase" timesteps.
    """
    # TODO: figure out formula for how com_vel relates to foot trajectories...
    # is SLIP model necessary?
    com_vel = 0.02 

    t = np.arange(phase)

    m = stride / phase

    ts = np.array([0, phase / 2, phase - 1])
    zs = np.array([0, peak, 0])
    spline = CubicSpline(ts, zs)

    x = m * t
    z = spline(t)

    # left foot steps first
    x_l = np.concatenate((x, np.ones(phase) * x[-1]))
    y_l = np.ones(phase * 2) * -stance
    z_l = np.concatenate((z, np.zeros(phase)))

    # then right foot
    x_r = np.concatenate((np.ones(phase) * x[0] + stride / 2, x + stride / 2))
    y_r = np.ones(phase * 2) * stance
    z_r = np.concatenate((np.zeros(phase), z))

    x_com = np.cumsum(np.ones(phase * 2) * com_vel)
    y_com = np.zeros(phase * 2)
    z_com = np.ones(phase * 2) * 1


    return np.column_stack([x_l, y_l, z_l, 
                       x_r, y_r, z_r, 
                       x_com, y_com, z_com])

spline_traj = get_trajectory(0.2, 0.0, 825)

print(spline_traj.shape)

(1650, 9)


In [5]:
import pickle
from cassie.cassiemujoco import pd_in_t

#with open("bslip_stepping_traj.pkl", "rb") as f:
#    bslip_data = pickle.load(f)

env = CassieEnv("walking")
env.render()
#input()
traj_qpos = []
for i in range(0, 1650):
    fpos_des = spline_traj[i, :]#torch.Tensor(bslip_data["foot"][i])

    lfoot = torch.Tensor(fpos_des[[0,2]]) - torch.Tensor([0, 0.99])
    rfoot = torch.Tensor(fpos_des[[3,5]]) - torch.Tensor([0, 0.99])
    
    qposl = model(lfoot).detach().numpy()
    qposr = model(rfoot).detach().numpy()
    
    #print(fpos_des)
    #qpos = qpos.detach().numpy()

    qpos = np.concatenate([fbpos, qposl, qposr])

    qpos[0] = 0#fpos_des[6]#bslip_data["com"][i][0]
    qpos[2] = 1.065#fpos_des[8]#bslip_data["com"][i][1] + 0.065 # 0.65 accounts for height of foot
    
    traj_qpos += [qpos]

    env.sim.set_qpos(qpos)
    env.sim.set_qvel(np.zeros(32))
    env.sim.step_pd(pd_in_t())
    
#     if(np.max(qpos - env.sim.qpos()) > 0.008):
#         print(np.max(qpos - env.sim.qpos()))
#         print(np.argmax(qpos - env.sim.qpos()))
#         import pdb; pdb.set_trace()

    env.render()

    time.sleep(1/240)

### 

In [6]:
print(len(traj_qpos))

# Now we gotta estimate the qvels using finite difference estimates of d qpos / dt

traj_qvel = []
for i in range(len(traj_qpos)):
    # note: index < 0 is valid because this is a periodic trajectory
    traj_qvel += [(traj_qpos[i] - traj_qpos[i - 1]) / (1/2000)]

trajectory = {"qpos": traj_qpos, "qvel": traj_qvel}

with open("cassie/trajectory/spline_stepping_traj2.pkl", "wb") as f:
    pickle.dump(trajectory, f)

1650


In [15]:
env = CassieEnv("walking")
env.render()

for i in range(len(traj_qpos)//60):
    env.sim.set_qpos(traj_qpos[i*60])
    env.sim.set_qvel(0*traj_qvel[i*60])

    for t in range(1000):
        env.sim.step_pd(pd_in_t())
        env.render()
        time.sleep(1/500)

